In [1]:
""" Import Package """
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import os
import random
from tqdm import tqdm
import wandb
import time
import copy
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import _LRScheduler
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations

""" Global variables """
DATA = "./data"
TRAIN_DATA_PATH = f"{DATA}/training.csv"
TEST_DATA_PATH = f"{DATA}/test.csv"
OUTPUT_PREDICTION = f"{DATA}/predictions.csv"

First, we want to find the relationthips among the features in the dataset in order to help us decide how to utilize the combination of features.

In [2]:
""" Visualize the relationships among features """
'''train_dataset = pd.read_csv(TRAIN_DATA_PATH)

features = train_dataset[['query_length', 'is_homepage', 'sig1', 'sig2', 
                          'sig3', 'sig4', 'sig5', 'sig6', 'sig7', 'sig8']]
sns.pairplot(features)
plt.show()'''

"train_dataset = pd.read_csv(TRAIN_DATA_PATH)\n\nfeatures = train_dataset[['query_length', 'is_homepage', 'sig1', 'sig2', \n                          'sig3', 'sig4', 'sig5', 'sig6', 'sig7', 'sig8']]\nsns.pairplot(features)\nplt.show()"

In [3]:
""" Load dataset """
class SearchEngineDataLoader(Dataset):
    """ Preprocess dataset """
    def __init__(self, file_path: str, train: bool, features: list, z_thresh=4.0):
        self.data = pd.read_csv(file_path)
        self.features = self.data[features].values
        if train:
            self.labels = self.data['relevance'].values
        else:
            self.labels = None
        
        pairwise_features = []
        '''for (f1, f2) in combinations([f for f in features if f.startswith('sig')], 2):
            self.data[f'{f1}_{f2}'] = self.data[f1] * self.data[f2]
            pairwise_features.append(f'{f1}_{f2}')'''
        
        # combine original and new features
        self.features_list = features + pairwise_features
        self.features = self.data[self.features_list].values

        # remove outliers
        if train == True:
            self.features, self.labels = self.remove_outliers(self.features, self.labels, z_thresh)

        # standardize data 
        scaler = StandardScaler()
        self.features = scaler.fit_transform(self.features)
    
    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        if self.labels is not None:
            label = torch.tensor(self.labels[idx], dtype=torch.float32).unsqueeze(0)
            return features, label
        else:
            return features
    
    def remove_outliers(self, features, labels, z_thresh):
        z_scores = np.abs((features - features.mean(axis=0)) / features.std(axis=0))
        mask = (z_scores < z_thresh).all(axis=1)
        features = features[mask]
        if labels is not None:
            labels = labels[mask]
        return features, labels
    
    def get_raw_feature(self, feature_name):
        """Method to access raw feature data"""
        return self.data[feature_name].values


def get_dataset(train_file_path: str,
                test_file_path: str,
                features: list):
    """ Create Dataset """
    train_dataset = SearchEngineDataLoader(train_file_path,
                                           train=True,
                                           features=features)
    test_dataset = SearchEngineDataLoader(test_file_path,
                                          train=False,
                                          features=features)
    return train_dataset, test_dataset


def get_dataloader(train_dataset: SearchEngineDataLoader,
                   test_dataset: SearchEngineDataLoader,
                   batch_size: int, 
                   val_proportion: float = 0,
                   pin_memory: bool = True,
                   shuffle: bool = True,
                   seed: int = 1) -> tuple[DataLoader, DataLoader, int, int]:
    """ Create Dataloader and return in_channels and num_classes """
    in_channels = train_dataset.features.shape[1]
    num_classes = 1  # Assuming binary classification

    # Split into train and validation sets if val_proportion > 0
    if val_proportion > 0:
        val_size = int(len(train_dataset) * val_proportion)
        train_size = len(train_dataset) - val_size
        generator = torch.Generator().manual_seed(seed)
        train_dataset, val_dataset = random_split(train_dataset, 
                                                  [train_size, val_size], 
                                                  generator=generator)
    else:
        train_dataset = train_dataset
        val_dataset = None
    
    # Create DataLoaders
    train_loader = DataLoader(train_dataset, 
                              batch_size=batch_size, 
                              shuffle=shuffle, 
                              num_workers=0, 
                              pin_memory=pin_memory)
    
    if val_dataset is not None:
        val_loader = DataLoader(val_dataset, 
                                batch_size=batch_size, 
                                shuffle=False, 
                                num_workers=0, 
                                pin_memory=pin_memory)
    else:
        val_loader = None
    
    test_loader = DataLoader(test_dataset, 
                             batch_size=batch_size, 
                             shuffle=False, 
                             num_workers=0, 
                             pin_memory=pin_memory)

    return train_loader, val_loader, test_loader, in_channels, num_classes


features = ['query_length', 'is_homepage', 'sig1', 'sig2', 
            'sig3', 'sig4', 'sig5', 'sig6', 'sig7', 'sig8']
batch_size = 1024

train_dataset, test_dataset = get_dataset(TRAIN_DATA_PATH,
                                          TEST_DATA_PATH,
                                          features)

train_loader, val_loader, test_loader, in_channels, num_class = get_dataloader(train_dataset=train_dataset,
                                                                               test_dataset=test_dataset,
                                                                               batch_size=batch_size,
                                                                               val_proportion=0.1,
                                                                               pin_memory=True,
                                                                               shuffle=True,
                                                                               seed=1,)
for feature, label in train_loader:
    print(feature.shape)
    print(label.shape)
    break
print(in_channels)
print(train_dataset.features.shape)

torch.Size([1024, 10])
torch.Size([1024, 1])
10
(77685, 10)


In [4]:
""" Modeling using Neural Network """
def torch_set_random_seed(seed: int = 1) -> None:
    """ Set random seed for reproducible usage """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


class WarmUpLR(_LRScheduler):
    # The fuction is adapted from https://github.com/weiaicunzai/pytorch-cifar100/blob/master/utils.py
    # author: baiyu
    """ Warmup_training learning rate scheduler """
    def __init__(self, optimizer, total_iters, last_epoch=-1):

        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """ Fpr first m batches, and set the learning rate to base_lr * m / total_iters """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]


def train(model: nn.Module, 
          train_loader: DataLoader, 
          loss_function: nn.Module,
          optimizer: optim.Optimizer,
          lr_scheduler_warmup: WarmUpLR,
          warm: int,
          epoch: int,
          device: str) -> float:
    """ Train model and save using early stop on test dataset """
    model.train()
    train_loss = 0.0
    for features, labels in train_loader:
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        if epoch <= warm:
            lr_scheduler_warmup.step()

    train_loss /= len(train_loader)
    return train_loss


@torch.no_grad()
def evaluate(model: nn.Module,
             eval_loader: DataLoader, 
             loss_function: nn.Module, 
             threshold: float,
             device: str) -> tuple[float, float]:
    """ Evaluate model """
    model.eval()
    correct = 0.0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for features, labels in eval_loader:
            features = features.to(device)
            labels = labels.to(device)

            outputs = model(features)
            loss = loss_function(outputs, labels)
            total_loss += loss.item()
            
            predicted = (torch.sigmoid(outputs) > threshold).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    accuracy = correct / total
    average_loss = total_loss / len(eval_loader)
    return accuracy, average_loss


""" Train model on dataloader """
model_name = "resnet34"
total_epoch = 50
initial_lr = 0
min_lr = 0
random_seed = 1
split_seed = 1
warm = 1

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Running on device: {device}")

output_dir = "output_models"
overall_best_acc = 0
optimal_initial_lr = 0
optimal_min_lr = 0

for threshold in [0.5]:
    for initial_lr in [1e-1]:
        for min_lr in [1e-3]:
            best_acc = 0.0
            for split_seed in [1, 5, 10, 20, 50, 100, 120, 150, 180, 200]:
                torch_set_random_seed(random_seed)
                hyperparams_config = {
                    "model": model_name,
                    "epoch": total_epoch,
                    "initial_lr": initial_lr,
                    "min_lr": min_lr,
                    "random_seed": random_seed
                }
                wandb.init(
                    project=f"STATS-Project-new-{model_name}",
                    name=f"{int(time.time())}",
                    id=str(int(time.time())),
                    config=hyperparams_config,
                    mode='online'
                )
                train_loader, val_loader, test_loader, in_channels, num_class = get_dataloader(train_dataset=train_dataset,
                                                                                                test_dataset=test_dataset,
                                                                                                batch_size=batch_size,
                                                                                                val_proportion=0.375,
                                                                                                pin_memory=True,
                                                                                                shuffle=True,
                                                                                                seed=split_seed,)
                from resnet import resnet18, resnet34, resnet50, resnet101, resnet152
                from vgg import vgg11, vgg13, vgg16, vgg19
                from lenet import Lenet5
                model = resnet34(in_channels, num_class).to(device)

                loss_function = nn.BCEWithLogitsLoss()
                optimizer = optim.SGD(model.parameters(), lr=initial_lr, momentum=0.9, weight_decay=1e-2)
                lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                                    T_max=total_epoch - 5 - warm, 
                                                                    eta_min= min_lr,
                                                                    last_epoch=-1)
                iter_per_epoch = len(train_loader)
                lr_scheduler_warmup = WarmUpLR(optimizer, iter_per_epoch * warm)

                cur_best_acc = 0
                with tqdm(total=total_epoch, desc=f'Training', unit='epoch') as pbar:
                    for epoch in range(1, total_epoch + 1):
                        train_loss = train(model, 
                                            train_loader, 
                                            loss_function, 
                                            optimizer, 
                                            lr_scheduler_warmup,
                                            warm,
                                            epoch,
                                            device)
                        top1_acc, eval_loss = evaluate(model, 
                                                    val_loader, 
                                                    loss_function, 
                                                    threshold,
                                                    device)
                        if epoch <= total_epoch - 5 and epoch > warm:
                            lr_scheduler.step()

                        if cur_best_acc < top1_acc:
                            cur_best_acc = top1_acc
                            if best_acc < cur_best_acc:
                                best_acc = cur_best_acc
                                os.makedirs(f"{output_dir}", exist_ok=True)
                                output_pth = f"{output_dir}/best_model.pth"
                                print(f"Best model stored to {output_pth}")
                                torch.save(model, output_pth)
                        
                        for param_group in optimizer.param_groups:
                            lr = param_group['lr']
                        
                        wandb.log({"epoch": epoch, "train_loss": train_loss, 'lr': lr,
                                   "top1_acc": top1_acc, "eval_loss": eval_loss, 'threshold': threshold,
                                   "best top1 acc": cur_best_acc})
                        
                        pbar.set_postfix({'Train loss': train_loss, 'overall_best_acc': overall_best_acc, 'Best top1 acc': cur_best_acc, 'Top1 acc': top1_acc})
                        pbar.update(1)
                
                wandb.finish()
                print(f"Seed {split_seed} has best_acc: {cur_best_acc}")
                overall_best_acc += best_acc

print(overall_best_acc / 10)
print(best_acc)
print(optimal_initial_lr)
print(optimal_min_lr)
print(total_epoch)


Running on device: cuda


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: wangboyao-02 (beryex). Use `wandb login --relogin` to force relogin


Training:   0%|          | 0/50 [00:00<?, ?epoch/s]

Best model stored to output_models/best_model.pth


Training:  14%|█▍        | 7/50 [00:08<00:50,  1.18s/epoch, Train loss=0.62, overall_best_acc=0, Best top1 acc=0.65, Top1 acc=0.621] 

...

best top1 acc,▁▁▁▁▁▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval_loss,▆██▇▆▄▂▂▂▃▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁
lr,███████▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
threshold,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
top1_acc,▄▁▁▁▃▇▇▇▇▇█▇▇▇▇▇▇█▇▇▇▇▇█▇███████████████
train_loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
best top1 acc,0.66716
epoch,50
eval_loss,0.60928
lr,0.001


Seed 180 has best_acc: 0.667158696920806


Training: 100%|██████████| 50/50 [00:58<00:00,  1.16s/epoch, Train loss=0.605, overall_best_acc=5.99, Best top1 acc=0.662, Top1 acc=0.661]


best top1 acc,▁▄▄▄████████████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval_loss,█▇▇▇▅▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,███████▇▇▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
threshold,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
top1_acc,▁▄▃▃█▅█▇██████▇██▇███▇███▇██████████████
train_loss,█▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
best top1 acc,0.66173
epoch,50
eval_loss,0.61132
lr,0.001


Seed 200 has best_acc: 0.6617349215612235
0.665456043390203
0.667158696920806
0
0
50


In [5]:
""" Predict on test dataset using the best model """
@torch.no_grad()
def predict_and_save_results(model, test_loader, output_file, threshold):
    model.eval() 
    results = []

    with torch.no_grad():
        for idx, features in enumerate(test_loader):
            features = features.to(device)
            outputs = model(features)
            predictions = (torch.sigmoid(outputs) > threshold).float()

            for idx2, prediction in enumerate(predictions):
                sample_id = str(int(test_loader.dataset.data.iloc[idx*batch_size+idx2]['query_id'])) + str(int(test_loader.dataset.data.iloc[idx*batch_size+idx2]['url_id']))
                results.append({'id': sample_id, 'relevance': int(prediction)})

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)

best_model = torch.load(f"{output_dir}/best_model.pth").to(device)
predict_and_save_results(best_model, test_loader, OUTPUT_PREDICTION, threshold)